<a href="https://colab.research.google.com/github/JonPaulBIlbao/GitHub-Test/blob/master/Polig_linguistico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

path = "/content/drive/Shareddrives/(GDD) Unidad de Gobierno del dato/Proyectos/Politica ling/2023_24"

os.chdir(path)
print("✅ Current working directory:", os.getcwd())


✅ Current working directory: /content/drive/Shareddrives/(GDD) Unidad de Gobierno del dato/Proyectos/Politica ling/2023_24


In [4]:
# --- ACTIVAR PAQUETES (equivalentes a R: readxl, dplyr, tidyr, writexl) ---
import pandas as pd
import numpy as np

In [5]:
# -------------------------------------------------------------------------
# LECTURA DE LA BASE DE DATOS
# -------------------------------------------------------------------------

# Introduce Excel CAU number (igual que cau_number <- "xxx.xlsx")
cau_number = "CAU-115658_ConsultasAnteriores.xlsx"

# Leer cada hoja (sheet) exactamente como en R
TITUL = pd.read_excel(cau_number, sheet_name="Titulaciones")
PROFE = pd.read_excel(cau_number, sheet_name="Profesores")
ASIGN = pd.read_excel(cau_number, sheet_name="Asignaturas")

# Mostrar las primeras filas para comprobar (opcional)
TITUL.head(), PROFE.head(), ASIGN.head()


(  CURSO_ACADEMICO              NIVEL_ESTUDIO  CAMPUS  \
 0         2023-24   Estudios propios - Grado  BILBAO   
 1         2023-24  Estudios propios - Máster  BILBAO   
 2         2023-24  Estudios propios - Máster  BILBAO   
 3         2023-24  Estudios propios - Máster  BILBAO   
 4         2023-24  Estudios propios - Máster  BILBAO   
 
                                 CENTRO  \
 0  122 - Instituto de Estudios de Ocio   
 1            140 - Facultad de Derecho   
 2            140 - Facultad de Derecho   
 3            140 - Facultad de Derecho   
 4            140 - Facultad de Derecho   
 
                                              ESTUDIO  \
 0  12201 - Titulado Universitario en Cultura y So...   
 1                 78151 - Experto en Derecho Digital   
 2  78195 - Experto en Ley Concursal. Edición 2023...   
 3  78196 - Experto en Contratos Públicos. Edición...   
 4               79533 - Program in International Law   
 
                                                 PLA

In [6]:
print(TITUL.info())
print(PROFE.info())
print(ASIGN.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   CURSO_ACADEMICO                 203 non-null    object 
 1   NIVEL_ESTUDIO                   203 non-null    object 
 2   CAMPUS                          203 non-null    object 
 3   CENTRO                          203 non-null    object 
 4   ESTUDIO                         203 non-null    object 
 5   PLAN                            203 non-null    object 
 6   TIPO_PLAN                       203 non-null    object 
 7   ESTUDIO_CONJUNTO                203 non-null    object 
 8   TIPO_ESTUDIO_CONJUNTO           68 non-null     object 
 9   DURACION_ESTUDIO                203 non-null    int64  
 10  RAMA                            203 non-null    object 
 11  NUM_CREDITOS_NECESARIOS         203 non-null    float64
 12  ALUMNOS                         203 

In [7]:
# ================================================================
#   IMPORTS
# ================================================================
import pandas as pd
import numpy as np

# ================================================================
#   CONFIGURACIÓN
# ================================================================
SUM_COLS = [
    "ALUMNOS","AL_MENOS_1_ASIG_CAST","AL_MENOS_1_ASIG_EUSK","AL_MENOS_1_ASIG_INGL","AL_MENOS_1_ASIG_OTROS",
    "ALUMNOS_NUEVOINGRESO","AL_MENOS_1_ASIG_CAST_NUEVOINGR","AL_MENOS_1_ASIG_EUSK_NUEVOINGR",
    "AL_MENOS_1_ASIG_INGL_NUEVOINGR","AL_MENOS_1_ASIG_OTROS_NUEINGR","ALUMNOS_CAV","ALUMNOS_NO_CAV"
]

REQUIRED_COLS = SUM_COLS + [
    "NIVEL_ESTUDIO","CAMPUS","CENTRO","ESTUDIO","PLAN",
    "TIPO_ESTUDIO_CONJUNTO","DURACION_ESTUDIO","RAMA","NUM_CREDITOS_NECESARIOS"
]

GROUP_KEYS = ["NIVEL_ESTUDIO","CAMPUS","CENTRO","ESTUDIO"]

# ================================================================
#   UTILIDADES
# ================================================================
def mode_scalar(s):
    """
    Devuelve el valor más frecuente del grupo.
    Equivalente a usar un único factor en R.
    """
    m = s.mode(dropna=False)
    return m.iloc[0] if not m.empty else np.nan


# ================================================================
#   LECTURA Y LIMPIEZA INICIAL
# ================================================================
def load_cau_excel(filepath):
    TITUL = pd.read_excel(filepath, sheet_name="Titulaciones")
    PROFE = pd.read_excel(filepath, sheet_name="Profesores")
    ASIGN = pd.read_excel(filepath, sheet_name="Asignaturas")

    # Convertir todo object → string limpio
    for df in (TITUL, PROFE, ASIGN):
        for col in df.select_dtypes(include=["object"]).columns:
            df[col] = df[col].astype(str).str.strip()

    return TITUL, PROFE, ASIGN


# ================================================================
#   FILTROS DE TITULACIONES
# ================================================================
def filter_titulaciones(TITUL):
    TITUL_G = TITUL[TITUL["NIVEL_ESTUDIO"] == "Grado"].copy()

    TITUL_G["NUM_CREDITOS_NECESARIOS"] = pd.to_numeric(
        TITUL_G["NUM_CREDITOS_NECESARIOS"], errors="coerce"
    )

    TITUL_G_240 = TITUL_G[TITUL_G["NUM_CREDITOS_NECESARIOS"] >= 240].copy()

    return TITUL_G_240


# ================================================================
#   VALIDACIÓN DE COLUMNAS
# ================================================================
def validate_columns(df):
    missing = [c for c in REQUIRED_COLS if c not in df.columns]
    if missing:
        raise KeyError(f"Faltan columnas en el Excel: {missing}")

    # Convertir numéricos donde toca
    for c in SUM_COLS + ["NUM_CREDITOS_NECESARIOS","DURACION_ESTUDIO"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")


# ================================================================
#   DETECTAR INCONSISTENCIAS INTERNAS
# ================================================================
def detect_inconsistencies(df):
    should_be_unique = ["TIPO_ESTUDIO_CONJUNTO","RAMA"]
    inconsistencies = {}

    grouped = df.groupby(GROUP_KEYS)

    for col in should_be_unique:
        tmp = grouped[col].nunique(dropna=False)
        bad = tmp[tmp > 1]
        if not bad.empty:
            inconsistencies[col] = bad

    if inconsistencies:
        print("⚠️ Inconsistencias internas detectadas:")
        for col, ser in inconsistencies.items():
            print(f" - {col}: {ser.shape[0]} grupos con >1 valor")
    else:
        print("✅ No se detectan inconsistencias internas.")

    return inconsistencies


# ================================================================
#   RESUMEN / AGRUPACIÓN EXACTAMENTE COMO EN R
# ================================================================
def summarize_titulaciones(df):

    agg_spec = {
        "PLAN": "nunique",
        "TIPO_ESTUDIO_CONJUNTO": mode_scalar,
        "DURACION_ESTUDIO": "max",
        "RAMA": mode_scalar,
        "NUM_CREDITOS_NECESARIOS": "max",
        "ALUMNOS": "sum",
        "AL_MENOS_1_ASIG_CAST": "sum",
        "AL_MENOS_1_ASIG_EUSK": "sum",
        "AL_MENOS_1_ASIG_INGL": "sum",
        "AL_MENOS_1_ASIG_OTROS": "sum",
        "ALUMNOS_NUEVOINGRESO": "sum",
        "AL_MENOS_1_ASIG_CAST_NUEVOINGR": "sum",
        "AL_MENOS_1_ASIG_EUSK_NUEVOINGR": "sum",
        "AL_MENOS_1_ASIG_INGL_NUEVOINGR": "sum",
        "AL_MENOS_1_ASIG_OTROS_NUEINGR": "sum",
        "ALUMNOS_CAV": "sum",
        "ALUMNOS_NO_CAV": "sum",
    }

    summary = (
        df.groupby(GROUP_KEYS, observed=False, as_index=False)
        .agg(agg_spec)
        .rename(columns={"PLAN": "N_planes"})
    )

    return summary


# ================================================================
#   CÁLCULO DE PORCENTAJES E INDICADORES
# ================================================================
def compute_indicators(df):

    df = df.copy()
    df = df[df["CAMPUS"].isin(["BILBAO","SAN SEBASTIÁN"])]

    df["PORC_EUSK"] = df["AL_MENOS_1_ASIG_EUSK"] / df["ALUMNOS"]
    df["PORC_INGL"] = df["AL_MENOS_1_ASIG_INGL"] / df["ALUMNOS"]

    # Global
    eusk_global = df["AL_MENOS_1_ASIG_EUSK"].sum() / df["ALUMNOS"].sum()
    eusk_cav    = df["AL_MENOS_1_ASIG_EUSK"].sum() / df["ALUMNOS_CAV"].sum()

    ing_global  = df["AL_MENOS_1_ASIG_INGL"].sum() / df["ALUMNOS"].sum()
    ing_cav     = df["AL_MENOS_1_ASIG_INGL"].sum() / df["ALUMNOS_CAV"].sum()

    # Por campus
    def campus_ratio(campus, col1, col2):
        sub = df[df["CAMPUS"] == campus]
        return sub[col1].sum() / sub[col2].sum()

    resumen = {
        "Euskera_global": eusk_global,
        "Euskera_global_CAV": eusk_cav,
        "Inglés_global": ing_global,
        "Inglés_global_CAV": ing_cav,
        "Euskera_Bilbao": campus_ratio("BILBAO","AL_MENOS_1_ASIG_EUSK","ALUMNOS"),
        "Euskera_Bilbao_CAV": campus_ratio("BILBAO","AL_MENOS_1_ASIG_EUSK","ALUMNOS_CAV"),
        "Euskera_SS": campus_ratio("SAN SEBASTIÁN","AL_MENOS_1_ASIG_EUSK","ALUMNOS"),
        "Euskera_SS_CAV": campus_ratio("SAN SEBASTIÁN","AL_MENOS_1_ASIG_EUSK","ALUMNOS_CAV"),
        "Inglés_Bilbao": campus_ratio("BILBAO","AL_MENOS_1_ASIG_INGL","ALUMNOS"),
        "Inglés_Bilbao_CAV": campus_ratio("BILBAO","AL_MENOS_1_ASIG_INGL","ALUMNOS_CAV"),
        "Inglés_SS": campus_ratio("SAN SEBASTIÁN","AL_MENOS_1_ASIG_INGL","ALUMNOS"),
        "Inglés_SS_CAV": campus_ratio("SAN_SEBASTIÁN" if "SAN_SEBASTIÁN" in df["CAMPUS"].unique() else "SAN SEBASTIÁN","AL_MENOS_1_ASIG_INGL","ALUMNOS_CAV"),
    }

    return df, resumen


# ================================================================
#   PIPELINE PRINCIPAL
# ================================================================
def run_pipeline(filepath, export_excel=False):

    print("📥 Cargando Excel CAU...")
    TITUL, PROFE, ASIGN = load_cau_excel(filepath)

    print("🔍 Filtrando titulaciones...")
    TITUL_G_240 = filter_titulaciones(TITUL)

    print("✅ Validando columnas...")
    validate_columns(TITUL_G_240)

    print("🔎 Detectando inconsistencias internas...")
    detect_inconsistencies(TITUL_G_240)

    print("📊 Resumen de titulaciones...")
    resumen = summarize_titulaciones(TITUL_G_240)

    print("📈 Calculando indicadores...")
    resumen_biss, indicadores = compute_indicators(resumen)

    print("\n✅ Pipeline completado con éxito.")
    print("Indicadores globales:")
    for k,v in indicadores.items():
        print(f" - {k}: {round(v,4)}")

    if export_excel:
        resumen.to_excel("TITUL_resumen.xlsx", index=False)
        resumen_biss.to_excel("TITUL_resumen_BISS.xlsx", index=False)
        print("📤 Exportación a Excel completada.")

    return resumen, resumen_biss, indicadores


# ================================================================
#   EJECUCIÓN DIRECTA
# ================================================================
# ejemplo:
# resumen, resumen_biss, indicadores = run_pipeline("CAU-115658_ConsultasAnteriores.xlsx")


In [8]:
resumen, resumen_biss, indicadores = run_pipeline("CAU-115658_ConsultasAnteriores.xlsx")

📥 Cargando Excel CAU...
🔍 Filtrando titulaciones...
✅ Validando columnas...
🔎 Detectando inconsistencias internas...
✅ No se detectan inconsistencias internas.
📊 Resumen de titulaciones...
📈 Calculando indicadores...

✅ Pipeline completado con éxito.
Indicadores globales:
 - Euskera_global: 0.2867
 - Euskera_global_CAV: 0.305
 - Inglés_global: 0.4206
 - Inglés_global_CAV: 0.4475
 - Euskera_Bilbao: 0.2247
 - Euskera_Bilbao_CAV: 0.2468
 - Euskera_SS: 0.4724
 - Euskera_SS_CAV: 0.4597
 - Inglés_Bilbao: 0.4491
 - Inglés_Bilbao_CAV: 0.4932
 - Inglés_SS: 0.3351
 - Inglés_SS_CAV: 0.3261


In [9]:
path = "/content/drive/Shareddrives/(GDD) Unidad de Gobierno del dato/Proyectos/Politica ling/2024_25/GDD-105.xlsx"

resumen, resumen_biss, indicadores = run_pipeline(path)



📥 Cargando Excel CAU...
🔍 Filtrando titulaciones...
✅ Validando columnas...
🔎 Detectando inconsistencias internas...
✅ No se detectan inconsistencias internas.
📊 Resumen de titulaciones...
📈 Calculando indicadores...

✅ Pipeline completado con éxito.
Indicadores globales:
 - Euskera_global: nan
 - Euskera_global_CAV: 0.0
 - Inglés_global: nan
 - Inglés_global_CAV: 0.0
 - Euskera_Bilbao: nan
 - Euskera_Bilbao_CAV: 0.0
 - Euskera_SS: nan
 - Euskera_SS_CAV: 0.0
 - Inglés_Bilbao: nan
 - Inglés_Bilbao_CAV: 0.0
 - Inglés_SS: nan
 - Inglés_SS_CAV: 0.0


/tmp/ipython-input-2822185562.py:150: RuntimeWarning: invalid value encountered in scalar divide
  eusk_global = df["AL_MENOS_1_ASIG_EUSK"].sum() / df["ALUMNOS"].sum()
/tmp/ipython-input-2822185562.py:153: RuntimeWarning: invalid value encountered in scalar divide
  ing_global  = df["AL_MENOS_1_ASIG_INGL"].sum() / df["ALUMNOS"].sum()
/tmp/ipython-input-2822185562.py:159: RuntimeWarning: invalid value encountered in scalar divide
  return sub[col1].sum() / sub[col2].sum()


In [11]:
# ================================================================
#   IMPORTS
# ================================================================
import pandas as pd
import numpy as np

# ================================================================
#   CONFIGURACIÓN
# ================================================================
SUM_COLS = [
    "ALUMNOS","AL_MENOS_1_ASIG_CAST","AL_MENOS_1_ASIG_EUSK","AL_MENOS_1_ASIG_INGL","AL_MENOS_1_ASIG_OTROS",
    "ALUMNOS_NUEVOINGRESO","AL_MENOS_1_ASIG_CAST_NUEVOINGR","AL_MENOS_1_ASIG_EUSK_NUEVOINGR",
    "AL_MENOS_1_ASIG_INGL_NUEVOINGR","AL_MENOS_1_ASIG_OTROS_NUEINGR","ALUMNOS_CAV","ALUMNOS_NO_CAV"
]

REQUIRED_COLS = SUM_COLS + [
    "NIVEL_ESTUDIO","CAMPUS","CENTRO","ESTUDIO","PLAN",
    "TIPO_ESTUDIO_CONJUNTO","DURACION_ESTUDIO","RAMA","NUM_CREDITOS_NECESARIOS"
]

GROUP_KEYS = ["NIVEL_ESTUDIO","CAMPUS","CENTRO","ESTUDIO"]

# ================================================================
#   UTILIDADES
# ================================================================
def mode_scalar(s):
    m = s.mode(dropna=False)
    return m.iloc[0] if not m.empty else np.nan


# ================================================================
#   LECTURA Y LIMPIEZA INICIAL
# ================================================================
def load_cau_excel(filepath):
    TITUL = pd.read_excel(filepath, sheet_name="Titulaciones")
    PROFE = pd.read_excel(filepath, sheet_name="Profesores")
    ASIGN = pd.read_excel(filepath, sheet_name="Asignaturas")

    for df in (TITUL, PROFE, ASIGN):
        for col in df.select_dtypes(include=["object"]).columns:
            df[col] = df[col].astype(str).str.strip()

    return TITUL, PROFE, ASIGN


# ================================================================
#   FILTROS DE TITULACIONES
# ================================================================
def filter_titulaciones(TITUL):
    TITUL_G = TITUL[TITUL["NIVEL_ESTUDIO"] == "Grado"].copy()
    TITUL_G["NUM_CREDITOS_NECESARIOS"] = pd.to_numeric(TITUL_G["NUM_CREDITOS_NECESARIOS"], errors="coerce")
    TITUL_G_240 = TITUL_G[TITUL_G["NUM_CREDITOS_NECESARIOS"] >= 240].copy()
    return TITUL_G_240


# ================================================================
#   VALIDACIÓN DE COLUMNAS
# ================================================================
def validate_columns(df):
    missing = [c for c in REQUIRED_COLS if c not in df.columns]
    if missing:
        raise KeyError(f"Faltan columnas en el Excel: {missing}")
    for c in SUM_COLS + ["NUM_CREDITOS_NECESARIOS","DURACION_ESTUDIO"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")


# ================================================================
#   DETECTAR INCONSISTENCIAS INTERNAS
# ================================================================
def detect_inconsistencies(df):
    should_be_unique = ["TIPO_ESTUDIO_CONJUNTO","RAMA"]
    inconsistencies = {}
    grouped = df.groupby(GROUP_KEYS)
    for col in should_be_unique:
        tmp = grouped[col].nunique(dropna=False)
        bad = tmp[tmp > 1]
        if not bad.empty:
            inconsistencies[col] = bad
    if inconsistencies:
        print("⚠️ Inconsistencias internas detectadas:")
        for col, ser in inconsistencies.items():
            print(f" - {col}: {ser.shape[0]} grupos con >1 valor")
    else:
        print("✅ No se detectan inconsistencias internas.")
    return inconsistencies


# ================================================================
#   RESUMEN / AGRUPACIÓN
# ================================================================
def summarize_titulaciones(df):
    agg_spec = {
        "PLAN": "nunique",
        "TIPO_ESTUDIO_CONJUNTO": mode_scalar,
        "DURACION_ESTUDIO": "max",
        "RAMA": mode_scalar,
        "NUM_CREDITOS_NECESARIOS": "max",
        "ALUMNOS": "sum",
        "AL_MENOS_1_ASIG_CAST": "sum",
        "AL_MENOS_1_ASIG_EUSK": "sum",
        "AL_MENOS_1_ASIG_INGL": "sum",
        "AL_MENOS_1_ASIG_OTROS": "sum",
        "ALUMNOS_NUEVOINGRESO": "sum",
        "AL_MENOS_1_ASIG_CAST_NUEVOINGR": "sum",
        "AL_MENOS_1_ASIG_EUSK_NUEVOINGR": "sum",
        "AL_MENOS_1_ASIG_INGL_NUEVOINGR": "sum",
        "AL_MENOS_1_ASIG_OTROS_NUEINGR": "sum",
        "ALUMNOS_CAV": "sum",
        "ALUMNOS_NO_CAV": "sum",
    }

    summary = (
        df.groupby(GROUP_KEYS, observed=False, as_index=False)
        .agg(agg_spec)
        .rename(columns={"PLAN": "N_planes"})
    )
    return summary


# ================================================================
#   CÁLCULO DE INDICADORES
# ================================================================
def compute_indicators(df):
    df = df.copy()
    df = df[df["CAMPUS"].isin(["BILBAO","SAN SEBASTIÁN"])]

    df["PORC_EUSK"] = df["AL_MENOS_1_ASIG_EUSK"] / df["ALUMNOS"]
    df["PORC_INGL"] = df["AL_MENOS_1_ASIG_INGL"] / df["ALUMNOS"]

    # --- globales ---
    eusk_global = df["AL_MENOS_1_ASIG_EUSK"].sum() / df["ALUMNOS"].sum()
    eusk_cav    = df["AL_MENOS_1_ASIG_EUSK"].sum() / df["ALUMNOS_CAV"].sum()
    ing_global  = df["AL_MENOS_1_ASIG_INGL"].sum() / df["ALUMNOS"].sum()
    ing_cav     = df["AL_MENOS_1_ASIG_INGL"].sum() / df["ALUMNOS_CAV"].sum()

    # --- campus ---
    def campus_ratio(campus, col1, col2):
        sub = df[df["CAMPUS"] == campus]
        return sub[col1].sum() / sub[col2].sum()

    resumen = {
        "Euskera_global": eusk_global,
        "Euskera_global_CAV": eusk_cav,
        "Inglés_global": ing_global,
        "Inglés_global_CAV": ing_cav,
        "Euskera_Bilbao": campus_ratio("BILBAO","AL_MENOS_1_ASIG_EUSK","ALUMNOS"),
        "Euskera_Bilbao_CAV": campus_ratio("BILBAO","AL_MENOS_1_ASIG_EUSK","ALUMNOS_CAV"),
        "Euskera_SS": campus_ratio("SAN SEBASTIÁN","AL_MENOS_1_ASIG_EUSK","ALUMNOS"),
        "Euskera_SS_CAV": campus_ratio("SAN SEBASTIÁN","AL_MENOS_1_ASIG_EUSK","ALUMNOS_CAV"),
        "Inglés_Bilbao": campus_ratio("BILBAO","AL_MENOS_1_ASIG_INGL","ALUMNOS"),
        "Inglés_Bilbao_CAV": campus_ratio("BILBAO","AL_MENOS_1_ASIG_INGL","ALUMNOS_CAV"),
        "Inglés_SS": campus_ratio("SAN SEBASTIÁN","AL_MENOS_1_ASIG_INGL","ALUMNOS"),
        "Inglés_SS_CAV": campus_ratio("SAN SEBASTIÁN","AL_MENOS_1_ASIG_INGL","ALUMNOS_CAV"),
    }

    # --- indicadores por ESTUDIO ---
    indicadores_estudio = (
        df.groupby(["CAMPUS","CENTRO","ESTUDIO"], as_index=False)
          .apply(lambda x: pd.Series({
              "EUSKERA_sobre_ALUMNOS": x["AL_MENOS_1_ASIG_EUSK"].sum()/x["ALUMNOS"].sum() if x["ALUMNOS"].sum()>0 else np.nan,
              "EUSKERA_sobre_CAV": x["AL_MENOS_1_ASIG_EUSK"].sum()/x["ALUMNOS_CAV"].sum() if x["ALUMNOS_CAV"].sum()>0 else np.nan,
              "INGLES_sobre_ALUMNOS": x["AL_MENOS_1_ASIG_INGL"].sum()/x["ALUMNOS"].sum() if x["ALUMNOS"].sum()>0 else np.nan,
              "INGLES_sobre_CAV": x["AL_MENOS_1_ASIG_INGL"].sum()/x["ALUMNOS_CAV"].sum() if x["ALUMNOS_CAV"].sum()>0 else np.nan
          }))
          .reset_index(drop=True)
    )

    return df, resumen, indicadores_estudio


# ================================================================
#   PIPELINE PRINCIPAL
# ================================================================
def run_pipeline(filepath, export_excel=False):
    print("📥 Cargando Excel CAU...")
    TITUL, PROFE, ASIGN = load_cau_excel(filepath)

    print("🔍 Filtrando titulaciones...")
    TITUL_G_240 = filter_titulaciones(TITUL)

    print("✅ Validando columnas...")
    validate_columns(TITUL_G_240)

    print("🔎 Detectando inconsistencias internas...")
    detect_inconsistencies(TITUL_G_240)

    print("📊 Resumen de titulaciones...")
    resumen = summarize_titulaciones(TITUL_G_240)

    print("📈 Calculando indicadores...")
    resumen_biss, indicadores, indicadores_estudio = compute_indicators(resumen)

    print("\n✅ Pipeline completado con éxito.")
    print("Indicadores globales:")
    for k,v in indicadores.items():
        print(f" - {k}: {round(v,4)}")

    if export_excel:
        resumen.to_excel("TITUL_resumen.xlsx", index=False)
        resumen_biss.to_excel("TITUL_resumen_BISS.xlsx", index=False)
        indicadores_estudio.to_excel("Indicadores_por_ESTUDIO.xlsx", index=False)
        print("📤 Exportación a Excel completada.")

    return resumen, resumen_biss, indicadores, indicadores_estudio


# ================================================================
#   EJECUCIÓN
# ================================================================
# ejemplo:
# resumen, resumen_biss, indicadores, indicadores_estudio = run_pipeline("CAU-115658_ConsultasAnteriores.xlsx", export_excel=True)


In [12]:
resumen, resumen_biss, indicadores, indicadores_estudio = run_pipeline("CAU-115658_ConsultasAnteriores.xlsx", export_excel=True)

📥 Cargando Excel CAU...
🔍 Filtrando titulaciones...
✅ Validando columnas...
🔎 Detectando inconsistencias internas...
✅ No se detectan inconsistencias internas.
📊 Resumen de titulaciones...
📈 Calculando indicadores...

✅ Pipeline completado con éxito.
Indicadores globales:
 - Euskera_global: 0.2867
 - Euskera_global_CAV: 0.305
 - Inglés_global: 0.4206
 - Inglés_global_CAV: 0.4475
 - Euskera_Bilbao: 0.2247
 - Euskera_Bilbao_CAV: 0.2468
 - Euskera_SS: 0.4724
 - Euskera_SS_CAV: 0.4597
 - Inglés_Bilbao: 0.4491
 - Inglés_Bilbao_CAV: 0.4932
 - Inglés_SS: 0.3351
 - Inglés_SS_CAV: 0.3261
📤 Exportación a Excel completada.


/tmp/ipython-input-2085842664.py:159: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
